# Spotify Recommendation System - Experimentation Notebook

This notebook provides a complete framework for training, evaluating, and visualizing a baseline collaborative filtering model for song recommendations.

## 1. Setup

Install and import necessary libraries. We add `matplotlib` for plotting.

In [ ]:
!pip install pandas scikit-learn tqdm matplotlib

In [ ]:
import pandas as pd
import numpy as np
import json
import os
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import matplotlib.pyplot as plt

## 2. Experiment Tracking Setup

We'll create a list to store the results of each experiment. This will allow us to easily compare runs and plot the outcomes.

In [ ]:
experiment_results = []

## 3. Core Functions (Data Loading, Evaluation)

These are the helper functions for loading data and calculating our evaluation metrics.

In [ ]:
def load_playlist_slice(path, slice_nums=[0]):
    """Loads one or more JSON slice files from the dataset."""
    all_playlists = []
    for slice_num in slice_nums:
        filename = f'mpd.slice.{slice_num*1000}-{(slice_num+1)*1000-1}.json'
        try:
            with open(os.path.join(path, filename)) as f:
                data = json.load(f)
                all_playlists.extend(data['playlists'])
        except FileNotFoundError:
            print(f"Warning: Slice file not found at {os.path.join(path, filename)}. Skipping.")
    return all_playlists

def precision_at_k(k, recommendations, holdout_items):
    recs_at_k = recommendations[:k]
    hits = len(set(recs_at_k) & set(holdout_items))
    return hits / k

def recall_at_k(k, recommendations, holdout_items):
    recs_at_k = recommendations[:k]
    hits = len(set(recs_at_k) & set(holdout_items))
    return hits / len(holdout_items) if len(holdout_items) > 0 else 0

## 4. Experiment Runner

This is the main function that encapsulates the entire process: data loading, preprocessing, training, and evaluation. It takes your parameters, runs the experiment, and stores the results.

In [ ]:
def run_experiment(slice_nums, n_components, k=10): #k here generates 10 songs
    """Runs a full experiment cycle with given parameters and stores the results."""
    
    # --- 1. Data Loading & Preprocessing ---
    print(f"Loading {len(slice_nums)} slice(s) of data...")
    data_path = '/Users/pabil/Downloads/spotify_million_playlist_dataset/data/'
    playlists = load_playlist_slice(data_path, slice_nums=slice_nums)
    if not playlists:
        print("No data loaded. Aborting experiment.")
        return

    all_tracks = [track['track_uri'] for p in playlists for track in p['tracks']]
    unique_tracks = sorted(list(set(all_tracks)))
    track_to_idx = {track: i for i, track in enumerate(unique_tracks)}
    idx_to_track = {i: track for track, i in track_to_idx.items()}
    pid_to_idx = {p['pid']: i for i, p in enumerate(playlists)}

    # Create train-test split
    rows, cols, test_set = [], [], {}
    for p in playlists:
        playlist_idx = pid_to_idx[p['pid']]
        tracks = [t['track_uri'] for t in p['tracks']]
        if len(tracks) > 5:
            np.random.shuffle(tracks)
            num_holdout = int(len(tracks) * 0.2)
            test_set[playlist_idx] = [uri for uri in tracks[-num_holdout:] if uri in track_to_idx]
            train_tracks = tracks[:-num_holdout]
        else:
            train_tracks = tracks
        for track_uri in train_tracks:
            if track_uri in track_to_idx:
                rows.append(playlist_idx)
                cols.append(track_to_idx[track_uri])

    interaction_matrix_train = csr_matrix((np.ones(len(rows)), (rows, cols)), shape=(len(playlists), len(unique_tracks)))

    # --- 2. Model Training ---
    print(f"Training SVD model with n_components={n_components}...")
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    playlist_embeddings = svd.fit_transform(interaction_matrix_train)
    track_embeddings = svd.components_.T

    # --- 3. Evaluation ---
    print(f"Evaluating model with k={k}...")
    knn = NearestNeighbors(n_neighbors=k*2, metric='cosine').fit(track_embeddings)
    avg_precision = 0
    avg_recall = 0
    eval_count = 0

    for user_idx, holdout_items in tqdm(test_set.items(), desc="Evaluating"):
        input_track_indices = interaction_matrix_train[user_idx].indices
        if len(input_track_indices) == 0 or len(holdout_items) == 0:
            continue

        playlist_vector = np.mean(track_embeddings[input_track_indices], axis=0).reshape(1, -1)
        distances, indices = knn.kneighbors(playlist_vector, n_neighbors=k + len(input_track_indices))
        
        recommendations = []
        train_uris = {idx_to_track[i] for i in input_track_indices}
        for idx in indices.flatten():
            rec_uri = idx_to_track[idx]
            if rec_uri not in train_uris:
                recommendations.append(rec_uri)
        
        avg_precision += precision_at_k(k, recommendations[:k], holdout_items)
        avg_recall += recall_at_k(k, recommendations[:k], holdout_items)
        eval_count += 1

    avg_precision /= eval_count
    avg_recall /= eval_count
    
    print(f"Precision@{k}: {avg_precision:.4f}, Recall@{k}: {avg_recall:.4f}")

    # --- 4. Store Results ---
    experiment_results.append({
        'slice_nums': slice_nums,
        'data_size': len(playlists),
        'n_components': n_components,
        f'precision_at_{k}': avg_precision,
        f'recall_at_{k}': avg_recall
    })
    print("--- Experiment Complete ---\n")

## 5. Run Experiments

Now you can easily run multiple experiments by calling the `run_experiment` function with different parameters. We'll loop through a few values for `n_components` to see how it affects performance.

In [ ]:
# Clear previous results before starting a new set of experiments
experiment_results = []

# --- Experiment Set 1: Train on 10,000 playlists ---
print("--- Starting experiments on 10,000 playlists ---")
slices_for_10k = list(range(10))  # Slices 0-9, each slice contains 1000 playlists
component_options = [50, 100, 150]

for n_comps in component_options:
    run_experiment(slice_nums=slices_for_10k, n_components=n_comps, k=10)

# --- Experiment Set 2: Train on 100,000 playlists ---
# WARNING: This will take a very long time and use a lot of memory.
# You might want to run this overnight or on a more powerful machine.
##print("\n--- Starting experiments on 100,000 playlists ---")
##slices_for_100k = list(range(100))  # Slices 0-99

##for n_comps in component_options:
    ##run_experiment(slice_nums=slices_for_100k, n_components=n_comps, k=10)



## 6. Visualize Results

After the experiments are complete, we can convert our results list into a pandas DataFrame and plot the outcome to easily see the impact of our changes.

In [ ]:
if not experiment_results:
    print("No experiment results to plot. Please run some experiments first.")
else:
    results_df = pd.DataFrame(experiment_results)
    print("Experiment Results:")
    print(results_df)
    
    # Plotting Recall vs. n_components
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['n_components'], results_df['recall_at_10'], marker='o')
    plt.title('Model Performance vs. Number of Components')
    plt.xlabel('Number of Components')
    plt.ylabel('Recall@10')
    plt.grid(True)
    plt.xticks(results_df['n_components'])
    plt.show()